In [ ]:
%run ./common.ipynb
# %run ./JiraTransitions.ipynb
%run ./projectDetails.ipynb
import os

In [ ]:
g_Df = pd.DataFrame(columns=['key','FixVersion',
 'Priority',
 'Acceptance Criteria',
 'Severity Level',
 'Labels',
 'EpicLink',
 'Status',
 'Components',
 'IssueCreator',
 'creator.emailAddress',
 'creator.displayName',
 'SubTasks',
 'Reporter.name',
 'reporter.emailAddress',
 'reporter.displayName',
 'issueType',
 'projectKey',
 'projectName',
 'created_ts',
 'updated_ts',
 'ParentLink',
 'parent.key',
 'parent.Summary',
 'parent.priority',
 'parent.status',
 'summary',
 'StoryPoint',
 'Sprint',
 'Sprint_state',
 'Sprint_name',
 'Sprint_startDate',
 'Sprint_endDate',
 'Sprint_completeDate',
 'Sprint_activatedDate',
 'Sprint_goal',
  'Defect Type',
  'Bug Severity',
 'AssigneeName',
 'AssigneeEmailAddress',
 'AssigneeDisplayName',
 'ResolutionDescription',
 'ResolutionName'
 
])


logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

In [ ]:
issueLists = [
              {'issueType':"Portfolio Initiative",'partitionCnt':1},
              {'issueType':"Product Initiative", 'partitionCnt':1 },
              {'issueType':"Epic",'partitionCnt':3},
              {'issueType':"Story",'partitionCnt':20},
              {'issueType':"Task",'partitionCnt':3},
              {'issueType':"Sub-task",'partitionCnt':10},
              {'issueType':"Bug",'partitionCnt':5},
              {'issueType':"Incident",'partitionCnt':1},
              {'issueType':"Production Defects",'partitionCnt':1},
              {'issueType':"Defect",'partitionCnt':1},
              {'issueType':"Issue",'partitionCnt':1},
              {'issueType':"Test",'partitionCnt':20}
             ]

strings = projectList()

for issueList in issueLists:
    split_string_list = split_list(strings, issueList['partitionCnt'])

    for partition in split_string_list:
        df=fetch(', '.join(['"{}"'.format(value) for value in partition]),issueList['issueType'])
        if df is not None and not df.empty:
            g_Df = pd.concat([g_Df, df ], ignore_index=True)

In [ ]:
# g_Df=fetch("DNA","Story")
logger.info("Split Sprint struct to seperate fields...")
g_Df['Sprint'] = g_Df['Sprint'].apply(lambda x: convert_to_key_value(x))
g_Df['Sprint_state'] = g_Df['Sprint'].apply(lambda x: sprintvalue(x,'state') if x is not None else "NA" )
g_Df['Sprint_name']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'name') if x is not None else "NA" )
g_Df['Sprint_startDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'startDate') if x is not None else "NA" )
g_Df['Sprint_endDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'endDate') if x is not None else "NA" )
g_Df['Sprint_completeDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'completeDate') if x is not None else "NA" )
g_Df['Sprint_activatedDate']= g_Df['Sprint'].apply(lambda x: sprintvalue(x,'activatedDate') if x is not None else "NA" )
# g_Df['Sprint_goal']= epic_Df['Sprint'].apply(lambda x: sprintvalue(x,'goal') if x is not None else "NA" )
g_Df= g_Df.drop(columns=['Sprint'])

g_Df['FixVersion']= g_Df['FixVersion'].apply(lambda x: keyvalue(x,'name') if x is not None else "" )



In [ ]:
logger.info("Upload file to sharepoint...")


try:
    # Use os.path.join to safely construct the filepaths
    g_Df[g_Df['issueType'] == 'Portfolio Initiative'].to_csv(os.path.join(filepath, 'Portfolio_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Product Initiative'].to_csv(os.path.join(filepath, 'Product_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Epic'].to_csv(os.path.join(filepath, 'Epic.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Story'].to_csv(os.path.join(filepath, 'Story.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Task'].to_csv(os.path.join(filepath, 'Task.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Sub-task'].to_csv(os.path.join(filepath, 'Subtasks.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Test'].to_csv(os.path.join(filepath, 'Test.csv'), index=False)
    g_Df.loc[g_Df['issueType'].isin(['Bug','Epic','Incident','Production Defects','Defect','Issue'])].to_csv(os.path.join(filepath, 'Bug.csv'), index=False)

except Exception as e:  
    logger.error(f"Failed to upload files to {filepath}: {e}")
    pass


logger.info("Files uploaded")
logger.info("Script ended accumulating data from JIRA")
logger.info("=======================================")